In [1]:
JIRA_DOMAIN = "gremlininc.atlassian.net"

field

# Authentication

See README.md in this folder for how to setup authentication

In [2]:
import netrc

auth_file = netrc.netrc()
authTokens = auth_file.authenticators(JIRA_DOMAIN)

user_email = authTokens[0]
api_token = authTokens[2]

In [3]:
from jira import JIRA

jira_url = f"https://{JIRA_DOMAIN}/"

jira = JIRA(jira_url, basic_auth=(user_email, api_token))

# projects = jira.projects()  # Test Authentication

# Jira API Limits

Jira cloud limits issue searches to 100 results. We paginate over week long periods to collate all the issues from our desired
timeline.

In [14]:
import arrow

# start = arrow.get("2021-03-01")
start = arrow.get("2022-03-01")
end = arrow.utcnow()

jql_template = """
PROJECT = EN
AND created >= {}
AND created <= {}
AND statusCategory = Done
AND resolution = Done
"""

result_list = []

# If a result set has 99+ results, there's a good chance the API is limiting results and the date windows should be smaller.
class LimitedResultsException(Exception):
    pass

for r in arrow.Arrow.span_range('week', start, end):
    jql = jql_template.format(r[0].date(), r[1].date())
    results = jira.search_issues(jql, expand="changelog,renderedFields")
    
    if len(results) >= 99:
        raise LimitedResultsException()
    result_list.extend(results)

In [36]:
class WorkflowTransitionDAO:
    def __init__(self, timestamp, history_item):
        self._history_item = history_item
        
        self.field = history_item.field
        self.fieldtype = history_item.fieldtype
        
        self.timestamp = arrow.get(timestamp)
        
        self.from_state_id = getattr(history_item, "from")
        self.to_state_id = history_item.to
        
        self.from_state = history_item.fromString
        self.to_state = history_item.toString
        
    def __str__(self):
        return f"[{self.timestamp.date()}] '{self.from_state}'({self.from_state_id}) -> '{self.to_state}'({self.to_state_id})"


class WorkflowTransitionHistory(list):
    def boo(self):
        return "boo"

        
class Issue:
    def __init__(self, issue):
        self._issue = issue
        self.key = issue.key
        self.project = issue.fields.project
        self.created = issue.fields.created
        self.resolution_date = issue.fields.resolutiondate
        
        
    @property
    def state_transitions(self):
        history_items = WorkflowTransitionHistory()

        for h in self._issue.changelog.histories:
            for i in [x for x in h.items if x.field == 'status']:
                history_items.append(WorkflowTransitionDAO(h.created, i))
            # print(getattr(h, "created", None))
        
        return history_items
        
    def __str__(self):
        return f"[{self.key}]"
    
    def __repr__(self):
        return f"{self.key}"

In [37]:
issues = []

for r in result_list:
    issues.append(Issue(r))        

In [38]:
for i in issues:
    for st in i.state_transitions:
        print(st)

[2022-03-08] 'In Progress'(3) -> 'Done'(10001)
[2022-03-03] 'Backlog'(10090) -> 'In Progress'(3)
[2022-03-03] 'Backlog'(10090) -> 'Done'(10001)
[2022-04-22] 'In Progress'(3) -> 'Done'(10001)
[2022-04-14] 'Dev Ready'(10212) -> 'In Progress'(3)
[2022-04-11] 'Prioritized'(10211) -> 'Dev Ready'(10212)
[2022-04-11] 'Backlog'(10090) -> 'Prioritized'(10211)
[2022-04-07] 'Dev Review'(10143) -> 'Done'(10001)
[2022-04-04] 'In Progress'(3) -> 'Dev Review'(10143)
[2022-03-28] 'Dev Ready'(10212) -> 'In Progress'(3)
[2022-03-28] 'Backlog'(10090) -> 'Dev Ready'(10212)
[2022-03-04] 'In Progress'(3) -> 'Done'(10001)
[2022-03-02] 'Backlog'(10090) -> 'In Progress'(3)
[2022-03-04] 'Backlog'(10090) -> 'Done'(10001)
[2022-03-04] 'Backlog'(10090) -> 'Done'(10001)
[2022-03-04] 'Backlog'(10090) -> 'Done'(10001)
[2022-03-15] 'Backlog'(10090) -> 'Done'(10001)
[2022-03-03] 'Backlog'(10090) -> 'Done'(10001)
[2022-03-14] 'In Progress'(3) -> 'Done'(10001)
[2022-02-28] 'Backlog'(10090) -> 'In Progress'(3)
[2022-03-21

In [7]:
print(dao.state_transitions[0].__str__())

[2022-04-22] 'In Progress' -> 'Done'


In [41]:
s = jira.statuses()[0]


In [40]:
jira.statuscategories()

[<JIRA StatusCategory: key='undefined', name='No Category', id=1>,
 <JIRA StatusCategory: key='new', name='To Do', id=2>,
 <JIRA StatusCategory: key='indeterminate', name='In Progress', id=4>,
 <JIRA StatusCategory: key='done', name='Done', id=3>]

In [45]:
s.statusCategory

<JIRA StatusCategory: key='new', name='To Do', id=2>